In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
#import qgrid

In [2]:
tree

NameError: name 'tree' is not defined

In [37]:
wd = 'C://Users//17800//Desktop//python quant//Notes//pydata-book//examples'

## GroupBy Mechanics

In [4]:
df = pd.DataFrame({'key1':['a','a','b','b','a'],
                   'key2':['one','two','one','two','one'],
                   'data1':np.random.randn(5),
                   'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.157922,0.523474
1,a,two,0.585274,-0.169764
2,b,one,1.566488,0.075699
3,b,two,-0.488500,-0.684277
4,a,one,-0.382343,-1.136964


In [5]:
123

123

In [6]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [7]:
grouped.mean()

key1
a    0.015003
b    0.538994
Name: data1, dtype: float64

In [8]:
df['data1'].groupby([df['key1'],df['key2']]).mean()

key1  key2
a     one    -0.270133
      two     0.585274
b     one     1.566488
      two    -0.488500
Name: data1, dtype: float64

In [9]:
df['data1'].groupby([df['key1'],df['key2']]).mean().unstack()

key2,one,two
key1,,
a,-0.270133,0.585274
b,1.566488,-0.488500


In [10]:
#事实上，groupby的keys不必在df里，只需要保持长度相同即可
states = pd.Series(['Ohio','California','California','Ohio','Ohio'])
years = pd.Series([2005,2005,2006,2005,2006])

In [11]:
df['data1'].groupby([states,years]).mean()

California  2005    0.585274
            2006    1.566488
Ohio        2005   -0.323211
            2006   -0.382343
Name: data1, dtype: float64

In [12]:
#grouby操作对于一个DataFrame也是成立的
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.270133 -0.306745
     two   0.585274 -0.169764
b    one   1.566488  0.075699
     two  -0.488500 -0.684277

### Iterating Over Groups

In [13]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.157922  0.523474
1    a  two  0.585274 -0.169764
4    a  one -0.382343 -1.136964
b
  key1 key2     data1     data2
2    b  one  1.566488  0.075699
3    b  two -0.488500 -0.684277


In [14]:
for (k1,k2), group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.157922  0.523474
4    a  one -0.382343 -1.136964
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.585274 -0.169764
('b', 'one')
  key1 key2     data1     data2
2    b  one  1.566488  0.075699
('b', 'two')
  key1 key2   data1     data2
3    b  two -0.4885 -0.684277


In [15]:
dict(list(df.groupby('key1'))) #存为一个字典

{'a':   key1 key2     data1     data2
 0    a  one -0.157922  0.523474
 1    a  two  0.585274 -0.169764
 4    a  one -0.382343 -1.136964,
 'b':   key1 key2     data1     data2
 2    b  one  1.566488  0.075699
 3    b  two -0.488500 -0.684277}

In [16]:
dict(list(df.groupby(['key1','key2'])))

{('a',
  'one'):   key1 key2     data1     data2
 0    a  one -0.157922  0.523474
 4    a  one -0.382343 -1.136964,
 ('a',
  'two'):   key1 key2     data1     data2
 1    a  two  0.585274 -0.169764,
 ('b',
  'one'):   key1 key2     data1     data2
 2    b  one  1.566488  0.075699,
 ('b',
  'two'):   key1 key2   data1     data2
 3    b  two -0.4885 -0.684277}

In [17]:
dict(list(df.groupby(['key1','key2'])))[('a','one')] #字典中的元素为DataFrame

,key1,key2,data1,data2
0,a,one,-0.157922,0.523474
4,a,one,-0.382343,-1.136964


### Selecting a Column or Subset of Columns

In [18]:
df.groupby('key1')['data1'].mean()

key1
a    0.015003
b    0.538994
Name: data1, dtype: float64

### Grouping with Dicts and Series

Grouping information may exist in a form other than an array. Let's consider another example DataFrame.

In [19]:
people = pd.DataFrame(np.random.randn(5,5),
                      columns=['a','b','c','d','e'],
                      index=['Joe','Steve','Wes','Jim','Travis'])
people.iloc[2,[1,2]] = np.nan
people

,a,b,c,d,e
Joe,0.539832,-0.726161,-0.549548,-0.074499,1.594157
Steve,-1.082996,-0.951441,-0.680958,-0.071376,1.253452
Wes,0.870702,NaN,NaN,-0.030062,1.866252
Jim,0.616918,0.284227,0.500164,-0.341726,0.079480
Travis,0.545573,-0.011014,0.858335,-2.420278,0.268329


In [20]:
mapping = {'a': 'red',
           'b': 'red',
           'c': 'blue',
           'd': 'blue',
           'e': 'red',
           'f': 'orange'}

In [21]:
people.groupby(mapping,axis=1).sum()

,blue,red
Joe,-0.624047,1.407828
Steve,-0.752334,-0.780985
Wes,-0.030062,2.736955
Jim,0.158438,0.980625
Travis,-1.561943,0.802887


### Grouping with Functions

1.Using Python functions is a more generic way of defining a group mapping compared with a dict or Series.   
  
2.Any function passed as a group key will be called once per index vlaue per index value, with the return values being  used as the groups names.


In [22]:
people.groupby(len).sum()

,a,b,c,d,e
3,2.027452,-0.441934,-0.049384,-0.446287,3.539890
5,-1.082996,-0.951441,-0.680958,-0.071376,1.253452
6,0.545573,-0.011014,0.858335,-2.420278,0.268329


In [23]:
key_list = ['one','two','one','two','one']
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one  0.539832 -0.726161 -0.549548 -0.074499  1.594157
  two  0.616918  0.284227  0.500164 -0.341726  0.079480
5 two -1.082996 -0.951441 -0.680958 -0.071376  1.253452
6 one  0.545573 -0.011014  0.858335 -2.420278  0.268329

### Grouping by Index Levels

In [24]:
hier_df = pd.DataFrame(np.random.randn(4,5),
                        # defalut: range(4)
                        columns=[['US','US','US','JP','JP'],
                                 [1,3,5,1,3]])
hier_df.columns.names = ['cty','tenor']
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -1.603036  0.338198 -0.925113  1.202726  0.289839
1      1.217428 -1.812469  0.379187  1.321404 -0.284018
2      0.644005 -0.457323  0.540472  0.536724  1.163514
3     -1.041582  0.018771  0.519770  0.962345 -0.100928

In [25]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## Data Aggregation

### Multiple Function Application

In [26]:
df

,key1,key2,data1,data2
0,a,one,-0.157922,0.523474
1,a,two,0.585274,-0.169764
2,b,one,1.566488,0.075699
3,b,two,-0.488500,-0.684277
4,a,one,-0.382343,-1.136964


In [27]:
df.groupby('key1').quantile(0.9)

C:\Users\17800\AppData\Local\Temp/ipykernel_17104/194176330.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.quantile is deprecated. In a future version, a TypeError will be raised. Before calling .quantile, select only columns which should be valid for the function.
  df.groupby('key1').quantile(0.9)


,data1,data2
key1,,
a,0.436634,0.384826
b,1.360989,-0.000299


In [28]:
df.groupby(['key1','key2']).aggregate(lambda x:max(x)-min(x))

data1     data2
key1 key2                    
a    one   0.224421  1.660438
     two   0.000000  0.000000
b    one   0.000000  0.000000
     two   0.000000  0.000000

In [29]:
df.groupby(['key1','key2']).describe()

data1                                                             \
          count      mean      std       min       25%       50%       75%   
key1 key2                                                                    
a    one    2.0 -0.270133  0.15869 -0.382343 -0.326238 -0.270133 -0.214028   
     two    1.0  0.585274      NaN  0.585274  0.585274  0.585274  0.585274   
b    one    1.0  1.566488      NaN  1.566488  1.566488  1.566488  1.566488   
     two    1.0 -0.488500      NaN -0.488500 -0.488500 -0.488500 -0.488500   

                    data2                                                    \
                max count      mean       std       min       25%       50%   
key1 key2                                                                     
a    one  -0.157922   2.0 -0.306745  1.174107 -1.136964 -0.721855 -0.306745   
     two   0.585274   1.0 -0.169764       NaN -0.169764 -0.169764 -0.169764   
b    one   1.566488   1.0  0.075699       NaN  0.075699  0.075699  0.075699   
     two  -0.488500   1.0 -0.684277       NaN -0.684277 -0.684277 -0.684277   

                               
                75%       max  
key1 key2                      
a    one   0.108364  0.523474  
     two  -0.169764 -0.169764  
b    one   0.075699  0.075699  
     two  -0.684277 -0.684277

请注意下一条代码，虽然复杂，但极其好用

In [30]:
df.groupby(['key1','key2']).aggregate(['mean','std',lambda x:x.mean()]).rename(columns={'<lambda_0>':'mean_l'})

data1                        data2                    
               mean      std    mean_l      mean       std    mean_l
key1 key2                                                           
a    one  -0.270133  0.15869 -0.270133 -0.306745  1.174107 -0.306745
     two   0.585274      NaN  0.585274 -0.169764       NaN -0.169764
b    one   1.566488      NaN  1.566488  0.075699       NaN  0.075699
     two  -0.488500      NaN -0.488500 -0.684277       NaN -0.684277

In [31]:
#事实上，更换名字并没有那么难
df.groupby(['key1','key2']).aggregate([('mean'),('std'),('mean_l',lambda x:x.mean())])

data1                        data2                    
               mean      std    mean_l      mean       std    mean_l
key1 key2                                                           
a    one  -0.270133  0.15869 -0.270133 -0.306745  1.174107 -0.306745
     two   0.585274      NaN  0.585274 -0.169764       NaN -0.169764
b    one   1.566488      NaN  1.566488  0.075699       NaN  0.075699
     two  -0.488500      NaN -0.488500 -0.684277       NaN -0.684277

In [36]:
tips = pd.read_csv(wd+'//tips.csv')
tips

ImportError: Missing optional dependency 'fsspec'.  Use pip or conda to install fsspec.

In [33]:
tips.groupby(['day','smoker'])['tip'].aggregate(['mean','std'])

NameError: name 'tips' is not defined

In [34]:
tips.groupby(['day','smoker'])['tip'].aggregate(['mean','std',('range',lambda x:x.max()-x.min())])

NameError: name 'tips' is not defined

In [35]:
#事实上我们可以对不同变量做不同操作，这只需要输入一个字典即可
tips.groupby(['day','smoker'])[['tip','size']].aggregate({'tip':['min',('range',lambda x:x.max()-x.min())],
                                                        'size':['max']})

NameError: name 'tips' is not defined

### Returning Aggregated Data Without Row Indexes

In [ ]:
tips.groupby(['day','smoker'],as_index=False)[['tip','size']].aggregate({'tip':['min',('range',lambda x:x.max()-x.min())],
                                                        'size':['max']})

## Apply: General split-apply-combine

In [ ]:
tips.groupby('smoker').apply(lambda x:x.iloc[0:2,:])

### Example: Group Weighted Average

In [ ]:
df = pd.DataFrame({'category':['a','a','a','a','b','b','b','b'],
                   'data':np.random.randn(8),
                   'weights':np.random.rand(8)})
df

In [ ]:
df.groupby('category').apply(lambda x:np.average(x['data'],weights=x['weights']))

In [ ]:
close_px = pd.read_csv(wd+'//stock_px_2.csv', parse_dates=True,index_col=0)
close_px

In [ ]:
close_px.info()

### Example: Group-Wise Linear Regression

In [ ]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y,X).fit()
    return result

In [ ]:
rets = close_px.pct_change().dropna()

In [ ]:
rets.groupby(lambda x:x.year).apply(lambda x:sm.OLS(x['AAPL'],x['SPX'],).fit().params)

## Pivot Tables and Cross-Tabulation

### pivot_table

pivot_table  

|Arguments|Description|
|---|---|
|values|
|index|
|columns| 
|aggfunc| 
|fill_value| 
|dropna|
|margins|
||

In [ ]:
tips

In [ ]:
tips.pivot_table(['tip','size'],index=['time','day'],columns='smoker')

In [ ]:
tips.pivot_table(['tip','size'],index=['time','day'],columns='smoker',aggfunc=lambda x:len(x),margins=True,fill_value=0)

### Cross-Tabulation

$$ pd.crosstab $$

$$ a^5 b^5 $$

